#Semantic Search
## Case Study: Transform idle FAQ to Question Answering Model



In [11]:
!pip install sentence-transformers

In [12]:
import pandas as pd
import sklearn
import numpy as np

https://www.wwf.org.uk/

World Wide Fund for Nature
Non-governmental organization

In [13]:
wwf_faq=["I haven’t received my adoption pack. What should I do?",
         "How quickly will I receive my adoption pack?",
         "How can I renew my adoption?",
         "How do I change my address or other contact details?",
         "Can I adopt an animal if I don’t live in the UK?",
         "If I adopt an animal, will I be the only person who adopts that animal?",
"My pack doesn't contain a certicate",
"My adoption is a gift but won’t arrive on time. What can I do?",
"Can I pay for an adoption with a one-off payment?",
"Can I change the delivery address for my adoption pack after I’ve placed my order?",
"How long will my adoption last for?",
"How often will I receive updates about my adopted animal?",
"What animals do you have for adoption?",
"How can I nd out more information about my adopted animal?",
"How is my adoption money spent?",
"What is your refund policy?",
"An error has been made with my Direct Debit payment, can I receive a refund?",
"How do I change how you contact me?"]

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("quora-distilbert-base")

In [15]:
faq_embeddings = model.encode(wwf_faq)

In [16]:
test_questions=["What should be done, if the adoption pack did not reach to me?",
                " How fast is my adoption pack delivered to me?",
                "What should I do to renew my adoption?",
        "What should be done to change adress and contact details ?",
      "I live outside of the UK, Can I still adopt an animal?"]
test_q_emb= model.encode(test_questions)

In [17]:
from scipy.spatial.distance import cdist
for q, qe in zip(test_questions, test_q_emb):
    distances = cdist([qe], faq_embeddings, "cosine")[0]
    ind = np.argsort(distances, axis=0)[:3]
    print("\n Test Question: \n "+q)
    for i,(dis,text) in enumerate(zip(distances[ind], [wwf_faq[i] for i in ind])):
        print(dis,ind[i],text, sep="\t")


 Test Question: 
 What should be done, if the adoption pack did not reach to me?
0.14945798801495147	0	I haven’t received my adoption pack. What should I do?
0.24940208935707764	7	My adoption is a gift but won’t arrive on time. What can I do?
0.36697610450097407	1	How quickly will I receive my adoption pack?

 Test Question: 
  How fast is my adoption pack delivered to me?
0.1658239297264127	1	How quickly will I receive my adoption pack?
0.34704792078263125	0	I haven’t received my adoption pack. What should I do?
0.3511114575850516	7	My adoption is a gift but won’t arrive on time. What can I do?

 Test Question: 
 What should I do to renew my adoption?
0.04168240105290566	2	How can I renew my adoption?
0.29930176236683403	12	What animals do you have for adoption?
0.3014070037208386	0	I haven’t received my adoption pack. What should I do?

 Test Question: 
 What should be done to change adress and contact details ?
0.2766018540517977	3	How do I change my address or other contact detail

In [18]:
def get_best(query, K=3):
    query_embedding = model.encode([query])
    distances = cdist(query_embedding, faq_embeddings, "cosine")[0]
    ind = np.argsort(distances, axis=0)
    print("\n"+query)
    for c,i in list(zip(distances[ind],  ind))[:K]:
        print(c,wwf_faq[i], sep="\t")

In [19]:
get_best("How do I change my contact info?",3)


How do I change my contact info?
0.056767916805916196	How do I change my address or other contact details?
0.18566553083561854	How do I change how you contact me?
0.324083301839059	How can I renew my adoption?


In [20]:
get_best("How do I get my plane ticket if I bought it online?")


How do I get my plane ticket if I bought it online?
0.3594750344276968	How do I change how you contact me?
0.3680785260834347	How do I change my address or other contact details?
0.4306634265594925	My adoption is a gift but won’t arrive on time. What can I do?
